# Cleaning ETH and RTH Market Data
We combine extended trading hours (ETH) and regular trading hours (RTH) datasets into tidy
assets and option dataframes ready for downstream analysis.

## Dataset Overview
- **RTH vs ETH**: RTH covers the standard 09:30–16:00 ET NYSE session, while ETH captures the
  electronic session surrounding the close and open; both are needed to understand intraday shocks.
- **SPX**: The S&P 500 index level observed from cash equities.
- **VIX**: CBOE Volatility Index, capturing market-implied 30-day variance.
- **ES futures**: Front-month E-mini S&P 500 futures with bid/ask quotes that proxy overnight SPX moves.
- **SPX options**: Listed SPX index options with call/put strikes across expiries, quoted with bid/ask prices.

## Imports and Paths

In [1]:
from pathlib import Path

import pandas as pd

pd.options.display.width = 140
pd.options.display.max_columns = 20

PROJECT_ROOT = Path(".")
SESSION_FOLDERS = {"ETH": PROJECT_ROOT / "ETH", "RTH": PROJECT_ROOT / "RTH"}

## Helper Functions

In [2]:
# Helper routines: parsing CSVs, melting option quotes, and orchestrating session-level loads.
def parse_timestamped_csv(csv_path: Path) -> pd.DataFrame:
    """Load a CSV whose first column stores timestamps and return a tidy timestamp column."""
    frame = pd.read_csv(csv_path, index_col=0)
    frame.index = pd.to_datetime(frame.index)
    frame = frame.reset_index().rename(columns={"index": "timestamp"})
    return frame


def melt_option_quotes(frame: pd.DataFrame, value_name: str) -> pd.DataFrame:
    """Convert wide strike columns into long format for a single quote side."""
    melted = frame.melt(id_vars="timestamp", var_name="strike", value_name=value_name)
    melted["strike"] = pd.to_numeric(melted["strike"], errors="coerce")
    melted[value_name] = pd.to_numeric(melted[value_name], errors="coerce")
    return melted


def load_option_book(option_type: str) -> pd.DataFrame:
    """Load and merge bid/ask quotes for the requested option type across sessions."""
    records: list[pd.DataFrame] = []
    for session, folder in SESSION_FOLDERS.items():
        ask_files = sorted(folder.glob(f"*_{option_type}_ask_*.csv"))
        for ask_path in ask_files:
            stem_parts = ask_path.stem.split("_")
            quote_date = pd.to_datetime(stem_parts[0])
            expiry = pd.to_datetime(stem_parts[-1])
            bid_path = ask_path.with_name(ask_path.name.replace("_ask_", "_bid_"))
            if not bid_path.exists():
                continue

            ask_frame = melt_option_quotes(parse_timestamped_csv(ask_path), "ask")
            bid_frame = melt_option_quotes(parse_timestamped_csv(bid_path), "bid")
            merged = pd.merge(
                ask_frame, bid_frame, on=["timestamp", "strike"], how="outer"
            )

            merged["expiry"] = expiry
            merged["session"] = session
            merged["mid"] = merged[["bid", "ask"]].mean(axis=1)
            # merged = merged.dropna(subset=["strike"])
            # merged = merged.dropna(subset=["bid", "ask"], how="all")

            records.append(merged)

    long_df = pd.concat(records, ignore_index=True)
    long_df = long_df.sort_values(["timestamp", "strike"]).reset_index(drop=True)
    return long_df


def load_assets() -> pd.DataFrame:
    """Load SPX/ES/VIX snapshots from both sessions."""
    frames: list[pd.DataFrame] = []
    for session, folder in SESSION_FOLDERS.items():
        for csv_path in sorted(folder.glob("*_assets.csv")):
            frame = parse_timestamped_csv(csv_path)
            frame["session"] = session
            frames.append(frame)

    assets = pd.concat(frames, ignore_index=True)
    assets = assets.astype(
        {
            "SPX": "Float64",
            "ES_BID": "Float64",
            "ES_ASK": "Float64",
            "VIX": "Float64",
            "session": "category",
        }
    )
    assets = assets.sort_values("timestamp").reset_index(drop=True)
    return assets

## Load Assets Across Sessions

In [3]:
# Load, clean, and combine SPX, ES, and VIX snapshots from ETH and RTH.
assets_df = load_assets()
assets_df.head()

/var/folders/p3/7gy013yx2hn3mns87ysh4z4w0000gn/T/ipykernel_21230/3265375810.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  assets = pd.concat(frames, ignore_index=True)


,timestamp,SPX,ES_BID,ES_ASK,VIX,session
0,2025-04-01 09:31:21.491248,5590.32,5631.75,5632.0,22.66,RTH
1,2025-04-01 09:31:24.535800,5590.7,5631.75,5632.0,22.66,RTH
2,2025-04-01 09:31:27.582698,5589.06,5629.5,5629.75,22.66,RTH
3,2025-04-01 09:31:30.632772,5588.66,5630.5,5630.75,22.66,RTH
4,2025-04-01 09:31:33.638817,5588.82,5629.75,5630.0,22.69,RTH


In [4]:
assets_df.dtypes

timestamp    datetime64[ns]
SPX                 Float64
ES_BID              Float64
ES_ASK              Float64
VIX                 Float64
session            category
dtype: object

## Load Call Option Quotes

In [ ]:
# Build the consolidated call option dataframe with bid/ask/mid quotes.
call_options_df = load_option_book("call")
call_options_df.head()

## Load Put Option Quotes

In [ ]:
# Build the consolidated put option dataframe with bid/ask/mid quotes.
put_options_df = load_option_book("put")
put_options_df.head()